## this notebook is for our "hack" of ExoTransmit

**We're producing transit spectra for 2 different systems: HATS-6 b around the Sun, and an "M-dwarf" transiting the same Sun. We then multiply out the stellar radius in the transit spectra for each, so we can then divide the planet spectra by the M-dwarf spectra.**

Notes: all the spectra with ExoTransmit are calculated on the exact same wavelength grid of $0.3 - 30 \mu \text{m}$, so I use the same wavelength variable throughout. Also, ExoTransmit outputs in percent, but I've converted to ppm where needed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from smoothing import adap_smooth # my function

In [2]:
# Can be found in kipping/Exo_Transmit/Spectra
# as of 19 Jan 2024
spectra_path = '/Users/coffey/Downloads/kipping/Exo_Transmit/Spectra'

### M dwarf: HATS-6
**T-P profile**                : 3000 K (actually 3700 K)  
**EOS**                        : 1X_solar_gas  
**Planet g (m/s^2)**           : 481.95  
**Planet R (m)**               : 3.9655e+8  
**Star R (m)**                 : 6.957e+8  
**Pressure of cloud top (Pa)** : 0.0  
**Rayleigh scattering factor** : 0.0

**Chemistry** : VO, TiO, H2O, Na, K, CH4, CO2, CO, C2H2, C2H4, no collision induced, no scattering  

These parameters were copied from the TRAPPIST-1 parameters (just for now).

In [3]:
h6_spectrum = np.loadtxt(f'{spectra_path}/transmission_hats6.dat', skiprows = 2).T #full 0.3 - 30 um
wave    = h6_spectrum[0][:2900] * 1e6 # microns
h6_spec = h6_spectrum[1][:2900] / 100 # convert from %

### HATS-6 b
**T-P profile**                : 700 K  
**EOS**                        : 1X_solar_gas  
**Planet g (m/s^2)**           : 7.941  
**Planet R (m)**               : 7.118e+6  
**Star R (m)**                 : 6.957e+8  
**Pressure of cloud top (Pa)** : 0.0  
**Rayleigh scattering factor** : 0.0

**Chemistry** : VO, TiO, H2O, Na, K, no collision induced, no scattering  

These parameters were copied from the TRAPPIST-1 b parameters (just for now).

In [4]:
# using ExoTransmit
h6b_spectrum = np.loadtxt(f'{spectra_path}/transmission_hats6b.dat', skiprows = 2).T
h6b_spec     = h6b_spectrum[1][:2900] / 100 # convert from %

In [5]:
# flat spectrum = no atmosphere
Rs   = 6.957e8  # Sun
Rh6b = 7.118e6  # HATS-6 b
Rh6  = 8.2927e7 # HATS-6

h6b_depth     = (Rh6b / Rs)**2
flat_h6b_spec = np.repeat(h6b_depth, len(h6_spec))

In [6]:
# for comparison, h6b around h6 w/ ExoTransmit
h6b_og_spectrum  = np.loadtxt(f'{spectra_path}/transmission_h6b_around_h6_noaerosols.dat', skiprows = 2).T
h6b_og_spec      = h6b_og_spectrum[1][:2900] * 1e4

FileNotFoundError: /Users/coffey/Downloads/kipping/Exo_Transmit/Spectra/transmission_h6b_around_h6_noaerosols.dat not found.

In [ ]:
# for plotting (dotted line)
h6bh6_fixed_depth = (Rh6b/Rh6)**2 * 1e6 # fixed transit depth of h6b around h6

### Hacking time

In [ ]:
# Multiplying out the Sun's radius
h6b_spec_noRs      = np.sqrt(h6b_spec) * Rs
h6_spec_noRs       = np.sqrt(h6_spec) * Rs
flat_h6b_spec_noRs = np.sqrt(flat_h6b_spec) * Rs

In [ ]:
# combining the spectra for our new (Rp/Rs)^2
hacked_spec      = (h6b_spec_noRs / h6_spec_noRs)**2 * 1e6      # ppm
flat_hacked_spec = (flat_h6b_spec_noRs / h6_spec_noRs)**2 * 1e6 # ppm

In [ ]:
# smoothing to JWST resolution (R ~ 100 for prism)
smooth_hacked_spec = adap_smooth(wave, hacked_spec, R = 100)
smooth_flat_hacked_spec = adap_smooth(wave, flat_hacked_spec, R = 100)

smooth_h6b_spec = adap_smooth(wave, h6b_spec, R = 100)
smooth_h6_spec = adap_smooth(wave, h6_spec, R = 100)
smooth_og_spec = adap_smooth(wave, h6b_og_spec, R = 100)


In [ ]:
# added a couple invisible axis elements to give space b/w last two plots while 
# closing the space between the first two
fig, axs = plt.subplots(6, 1, figsize = (9,12), height_ratios = [1,1,0.2,1,0.1,1], sharex = True)
fig.subplots_adjust(hspace = 0)

# Spectra
axs[0].plot(wave, smooth_og_spec, c = 'red')
axs[0].plot(wave, smooth_hacked_spec, c = 'blue')
axs[1].plot(wave, smooth_flat_hacked_spec, c = 'seagreen')
axs[3].plot(wave, np.array(smooth_h6b_spec) * 1e6, c = 'black')
axs[5].plot(wave, np.array(smooth_h6_spec) * 1e6, c = 'black')
fig.legend(['orig', 'hack', 'flat hack'], loc = (0.805,0.865))

# Broken axis
axs[0].set_ylim(h6bh6_fixed_depth, None)
axs[1].set_ylim(None, h6bh6_fixed_depth)
axs[0].spines.bottom.set_visible(False)
axs[1].spines.top.set_visible(False)
axs[0].xaxis.tick_top()
axs[1].tick_params(labeltop=True)  # don't put tick labels at the top
axs[1].xaxis.tick_bottom()
axs[1].xaxis.grid(True, which='minor')
axs[2].set_visible(False)
axs[4].set_visible(False)

# Slanted lines
d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], ms=12,ls="none", c='k', mec='k', mew=1, clip_on=False)
axs[0].plot([0, 1], [0, 0], transform=axs[0].transAxes, **kwargs)
axs[1].plot([0, 1], [1, 1], transform=axs[1].transAxes, **kwargs)

# Plotting fixed transit depth to compare
axs[0].axhline(y = h6bh6_fixed_depth, c = 'black', ls = 'dotted', lw = 3.5)

# Labels
fig.suptitle('HATS-6 b orbiting HATS-6', y = 0.915)
fig.text(0.04, 0.7, 'Transit depth (ppm)', va='center', rotation='vertical')
fig.text(0.65, 0.45, r'$(R_{h6b}/R_{\odot})^2$', fontsize = 14, bbox=dict(boxstyle='round',ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),))
fig.text(0.65, 0.25, r'$(R_{h6}/R_{\odot})^2$', fontsize = 14, bbox=dict(boxstyle='round',ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8),))
axs[5].set_xlabel('Wavelength (microns)')

## For pandexo, let's smooth the spectrum to the smallest resolution possible while still capturing the main features

In [ ]:
supersmooth_hacked_spec = adap_smooth(wave, hacked_spec, R = 25)
supersmooth_flat_hacked_spec = adap_smooth(wave, flat_hacked_spec, R = 25)

In [ ]:
# added a couple invisible axis elements to give space b/w last two plots while 
# closing the space between the first two
fig, axs = plt.subplots(2, 1, figsize = (9,6), height_ratios = [1,1], sharex = True)
fig.subplots_adjust(hspace = 0)

# Spectra
axs[0].plot(wave, smooth_hacked_spec, c = 'royalblue', alpha = 0.5, label = '_nolegend_')
axs[1].plot(wave, smooth_flat_hacked_spec, c = 'seagreen', alpha = 0.5, label = '_nolegend_')
axs[0].plot(wave, supersmooth_hacked_spec, c = 'royalblue')
axs[1].plot(wave, supersmooth_flat_hacked_spec, c = 'seagreen')
fig.legend(['hack','flat hack'], loc = (0.805,0.8))

# Broken axis
axs[0].set_ylim(h6bh6_fixed_depth, None)
axs[1].set_ylim(None, h6bh6_fixed_depth)
axs[0].spines.bottom.set_visible(False)
axs[1].spines.top.set_visible(False)
axs[0].xaxis.tick_top()
axs[1].tick_params(labeltop=True)  # don't put tick labels at the top
axs[1].xaxis.tick_bottom()
axs[1].xaxis.grid(True, which='minor')

# Slanted lines
d = .5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], ms=12,ls="none", c='k', mec='k', mew=1, clip_on=False)
axs[0].plot([0, 1], [0, 0], transform=axs[0].transAxes, **kwargs)
axs[1].plot([0, 1], [1, 1], transform=axs[1].transAxes, **kwargs)

# Plotting fixed transit depth to compare
axs[0].axhline(y = h6bh6_fixed_depth, c = 'black', ls = 'dotted', lw = 3.5)

# Labels
fig.suptitle('Super smooth', y = 0.915)
fig.text(0.04, 0.7, 'Transit depth (ppm)', va='center', rotation='vertical')
axs[1].set_xlabel('Wavelength (microns)')

## Write it to a text file

**Pandexo wants $(R_{p}/R_{s})^2$ in decimal units (NOT PPM). I'll give Pandexo the unsmoothed spectrum and will bin after, per their recommendation.**

In [ ]:
# Saving the array in a text file
pandexo_hacked_spec = np.array(hacked_spec) / 1e6
pandexo_flat_hacked_spec = np.array(flat_hacked_spec) / 1e6

hats6_hacked_spectrum = np.column_stack((wave, pandexo_hacked_spec))
np.savetxt("./pandexo_input_spectra/hats6_hacked_spectrum.txt", hats_hacked_spectrum)

flat_hats6_hacked_spectrum = np.column_stack((wave, pandexo_flat_hacked_spec))
np.savetxt("./pandexo_input_spectra/flat_hats6_hacked_spectrum.txt", flat_hats_hacked_spectrum)

## Adding on the pandexo errors
**The files are in the format x,y,err and the y and errors are already in ppm**

In [ ]:
pandexo_errs = np.loadtxt('./pandexo_output/pandexo_errs_25.txt').T

In [ ]:
fig, axs = plt.subplots(2, 1, figsize = (9,6), sharex = True)
fig.subplots_adjust(hspace = 0.1)
axs[1].set_xlim(0.75, 5.3)
axs[0].set_ylim(6000,9000)
axs[1].set_ylim(7000,7700)

# Spectra
axs[0].plot(wave, smooth_hacked_spec, c = 'royalblue', alpha = 0.5)
axs[1].plot(wave, smooth_flat_hacked_spec, c = 'seagreen', alpha = 0.5)
axs[0].errorbar(pandexo_errs[0], pandexo_errs[1], pandexo_errs[2], fmt = '.', c = 'royalblue')
axs[1].errorbar(pandexo_errs[0], pandexo_errs[1], pandexo_errs[2], fmt = '.', c = 'seagreen')


# Plotting fixed transit depth to compare
axs[0].axhline(y = h6bh6_fixed_depth, c = 'black', ls = 'dotted', lw = 2, label = '_nolegend_')

# Labels
fig.legend(['hack','pandexo', 'flat hack', 'pandexo'], loc = (0.775,0.75))
fig.suptitle('HATS-6 b orbiting HATS-6 (w/ Pandexo errors)', y = 0.915)
fig.text(0.04, 0.5, 'Transit depth (ppm)', va='center', rotation='vertical')
axs[1].set_xlabel('Wavelength (microns)')

In [ ]:
print(f'Median error for pandexo w/ hacked spectrum: {np.median(pandexo_errs[2]):.3f} ppm')